<a href="https://colab.research.google.com/github/xiaoyufan/nbme/blob/main/baseline_deberta.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# NBME Baseline DeBERTa

## Configurations

In [1]:
class CFG:
  device = 'tpu'
  input_dir = '/content/drive/MyDrive/CS7150 Deep Learning Project/Dataset/Preprocessed'
  mode = 'dev'
  model = 'microsoft/deberta-base'
  output_dir = '/content/drive/MyDrive/CS7150 Deep Learning Project/Dataset/Output'

## Packages

In [2]:
!pip install transformers
!pip install tokenizers

!pip install --force-reinstall git+https://github.com/xiaoyufan/nbme.git

!pip install cloud-tpu-client==0.10 torch==1.11.0 https://storage.googleapis.com/tpu-pytorch/wheels/colab/torch_xla-1.11-cp37-cp37m-linux_x86_64.whl

  Cloning https://github.com/xiaoyufan/nbme.git to /tmp/pip-req-build-8smsa22t
  Running command git clone -q https://github.com/xiaoyufan/nbme.git /tmp/pip-req-build-8smsa22t
  Created wheel for nbme: filename=nbme-1.0-py3-none-any.whl size=2308 sha256=115fab33b0775ca8a3b72f28d3da88fa8165d16c66697f43e75d541399ee175a
  Stored in directory: /tmp/pip-ephem-wheel-cache-ogiv7nyf/wheels/e2/dc/dd/b061e30220e414c0a096e4fbaddd405c1b526d1453d9a447e3
Successfully built nbme
  Attempting uninstall: nbme
    Found existing installation: nbme 1.0
    Uninstalling nbme-1.0:
      Successfully uninstalled nbme-1.0
  Using cached https://storage.googleapis.com/tpu-pytorch/wheels/colab/torch_xla-1.11-cp37-cp37m-linux_x86_64.whl (152.9 MB)


In [3]:
import pandas as pd
import torch
import torch.nn as nn

from nbme_utils import scoring
from transformers import AutoTokenizer

## Device

In [4]:
import os

device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

if CFG.device == 'tpu':
  assert os.environ['COLAB_TPU_ADDR'], 'Make sure to select TPU from Edit > Notebook settings > Hardware accelerator'

  import torch_xla
  import torch_xla.core.xla_model as xm

  device = xm.xla_device()

device

device(type='xla', index=1)

## Data Loading

In [5]:
train = pd.read_csv(f'{CFG.input_dir}/train.csv')
validate = pd.read_csv(f'{CFG.input_dir}/validate.csv')
train.shape, validate.shape

((11342, 8), (2958, 8))

In [6]:
if CFG.mode == 'dev':
  train = train.sample(n=800, random_state=0).reset_index(drop=True)
  validate = validate.sample(n=200, random_state=0).reset_index(drop=True)
train.shape, validate.shape

((800, 8), (200, 8))

## Tokenizer

In [7]:
tokenizer = AutoTokenizer.from_pretrained(CFG.model)
tokenizer.save_pretrained(f'{CFG.output_dir}/tokenizer')

('/content/drive/MyDrive/CS7150 Deep Learning Project/Dataset/Output/tokenizer/tokenizer_config.json',
 '/content/drive/MyDrive/CS7150 Deep Learning Project/Dataset/Output/tokenizer/special_tokens_map.json',
 '/content/drive/MyDrive/CS7150 Deep Learning Project/Dataset/Output/tokenizer/vocab.json',
 '/content/drive/MyDrive/CS7150 Deep Learning Project/Dataset/Output/tokenizer/merges.txt',
 '/content/drive/MyDrive/CS7150 Deep Learning Project/Dataset/Output/tokenizer/added_tokens.json',
 '/content/drive/MyDrive/CS7150 Deep Learning Project/Dataset/Output/tokenizer/tokenizer.json')

## Model